# Assignment 01: Credit Card Data Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

The file Credit Card Data.xlsx has 3 tabs with the following data:

Credit Card customer master data                                     
Spend (Transaction) data                                 
Card repayment information   

Read the data in the excel file using Pandas library:

In [2]:
# Credit Card customer master data
customer_data = pd.read_excel(r'data\Credit_Card_Data.xlsx', sheet_name='Customer Details')
print(customer_data.shape)
customer_data.sample(5)

(100, 8)


,No,Customer,Age,City,Product,Limit,Company,Segment
46,47,A47,16,CHENNAI,Platimum,1380000,C9,Normal Salary
82,83,A83,36,COCHIN,Platimum,10000,C6,Govt
36,37,A37,74,BANGALORE,Platimum,600018,C19,Normal Salary
20,21,A21,22,COCHIN,Platimum,780000,C21,Normal Salary
31,32,A32,48,COCHIN,Gold,1500000,C14,Self Employed


In [3]:
# Spend (Transaction) data
spend_data = pd.read_excel(r"data\Credit_Card_Data.xlsx", sheet_name='Spend')
print(spend_data.shape)
spend_data.sample(5)

(1500, 5)


,Sl No:,Customer,Month,Type,Amount
257,258,A6,2005-05-26,PETRO,485.320842
1198,1199,A43,2004-01-15,CLOTHES,20735.673519
1112,1113,A44,2004-04-06,BUS TICKET,41160.393574
1061,1062,A51,2006-03-31,FOOD,338.034409
1423,1424,A36,2005-02-01,AIR TICKET,374251.178232


In [4]:
# Card repayment information
repayment_data = pd.read_excel(r"data\Credit_Card_Data.xlsx", sheet_name='Repayment')
print(repayment_data.shape)
repayment_data.sample(5)

(807, 4)


,SL No:,Customer,Month,Amount
527,528,A48,2005-08-25,900.428611
325,326,A32,2005-06-12,4031.743695
349,350,A35,2004-02-14,11684.774696
662,663,A58,2005-09-26,93.291852
657,658,A58,2004-02-05,826430.961072


Check the data for any corrupt data, missing values, incorrect data types etc. Make the following substitution:  
Age of credit card customer can't be less than 18. In case age is less than 18, replace it with mean of age.

In [5]:
# Check for missing and data types
print("Missing values in Customer Data:")
print(customer_data.isna().sum().sum(), '\n')
print(customer_data.dtypes, '\n\n')
print("Missing values in Spend Data:")
print(spend_data.isna().sum().sum(), '\n')
print(spend_data.dtypes, '\n\n')
print("Missing values in Repayment Data:")
print(repayment_data.isna().sum().sum(), '\n')
print(repayment_data.dtypes)

Missing values in Customer Data:
0 

No           int64
Customer    object
Age          int64
City        object
Product     object
Limit        int64
Company     object
Segment     object
dtype: object 


Missing values in Spend Data:
0 

Sl No:               int64
Customer            object
Month       datetime64[ns]
Type                object
Amount             float64
dtype: object 


Missing values in Repayment Data:
4 

SL No:               int64
Customer            object
Month       datetime64[ns]
Amount             float64
dtype: object


In [6]:
repayment_data.dropna(inplace = True)
repayment_data.isna().sum().sum()

0

In [7]:
# Replace ages less than 18 with the mean age
mean_age = customer_data['Age'].mean()
customer_data['Age'] = customer_data['Age'].apply(lambda x: mean_age if x < 18 else x)

1. How many unique customers are there?

In [8]:
print(f"Number of unique customers: {customer_data['Customer'].nunique()}\n")

Number of unique customers: 100



2. How many spend categories are there?

In [9]:
print(f"Number of spend categories: {spend_data['Customer'].nunique()}\n")

Number of spend categories: 100



3. Which category has the highest average spend?

In [10]:
highest_avg_spend_category = spend_data.groupby('Type')['Amount'].mean().idxmax()
print(f"Category with the highest average spend: {highest_avg_spend_category}\n")

Category with the highest average spend: CAR



4. What is the average monthly spent by product categories?

In [11]:
avg_monthly_spend_by_category = spend_data.groupby('Type')['Amount'].mean()
print('Average monthly spend by product categories:')
avg_monthly_spend_by_category

Average monthly spend by product categories:


Type
AIR TICKET      254632.194082
AUTO             27320.763977
BIKE            210701.269489
BUS TICKET       12681.030717
CAMERA           21499.802175
CAR             409143.467018
CLOTHES          25140.160688
FOOD               341.168321
JEWELLERY       239218.687225
MOVIE TICKET      1875.298919
PETRO              549.483953
RENTAL           13106.511138
SANDALS           2516.628761
SHOPPING          7394.332039
TRAIN TICKET      1627.490364
Name: Amount, dtype: float64

5. Which customers are reaching 90% or more of their spending limit?

In [12]:
spend_limit_df = customer_data.merge(spend_data, on='Customer')
spend_limit_df['usage'] = spend_limit_df['Amount'] / spend_limit_df['Limit']
customers_above_90_percent = spend_limit_df[spend_limit_df['usage'] >= 0.9]['Customer']
print("Customers reaching 90% or more of their spending limit:")
customers_above_90_percent

Customers reaching 90% or more of their spending limit:


218     A13
286     A15
292     A16
707     A33
1218    A52
1227    A53
1442    A62
1446    A62
1448    A62
1465    A69
1495    A96
1498    A99
Name: Customer, dtype: object

6. Which city has the maximum number of spenders, each month? Is there a need to run campaigns in specific cities?

In [13]:
customer_spend_data = customer_data.merge(spend_data, on='Customer')
customer_spend_data['month'] = customer_spend_data['Month'].dt.month
city_monthly_spenders = customer_spend_data.groupby(['month', 'City'])['Customer'].nunique().reset_index()
city_max_spenders_per_month = city_monthly_spenders.sort_values(by=['month', 'Customer'], ascending=[True, False])
print('City with the maximum number of spenders each month:')
print(city_max_spenders_per_month.groupby('month').first())

City with the maximum number of spenders each month:
            City  Customer
month                     
1         COCHIN        16
2      BANGALORE        16
3         COCHIN        15
4      BANGALORE        12
5         COCHIN        13
6      BANGALORE         7
7      BANGALORE         9
8      BANGALORE         7
9         COCHIN         9
10     BANGALORE         8
11     BANGALORE        10
12        COCHIN         7


Need to run campaigns in cities Cochin and Bangalore as they consistently attract the maximum number of spenders.

7. Which age group spends the most?

In [14]:
bins = [18, 30, 40, 50, 60, 70, 120]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70+']
customer_data['age_group'] = pd.cut(customer_data['Age'], bins=bins, labels=labels)
age_group_spend = customer_data.merge(spend_data, on='Customer').groupby('age_group')['Amount'].sum()
print('Age Group that spends the most:')
age_group_spend.idxmax()

Age Group that spends the most:


'60-69'